<h2>Problem Statement : 

Make a Smart Home Assistant, similar to Alexa, which will be capable of turning on and off lights based on the voice commands given by the user. This will be handled by an LLM gathering voice inputs, and using langgraph to ensure the network structure of the house.
<br>#PART 1 : "What should I wear today?"


<br>#PART 2 : Full Smart home


</h2>

<h3>Python playbook

<h4> 1. Figuring out how to make speech-to-text work

In [ ]:
# Python program to translate
# speech to text and text to speech


import speech_recognition as sr
import pyttsx3 

# Initialize the recognizer 
r = sr.Recognizer() 

# Function to convert text to speech
def SpeakText(command):
    
    # Initialize the engine
    engine = pyttsx3.init()
    engine.say(command) 
    engine.runAndWait()
    
    
# Loop infinitely for user to speak

# while(1):    
#     try:
        
#         # use the microphone as source for input.
#         with sr.Microphone() as source2:
            
#             # wait for a second to let the recognizer adjust the energy threshold based on the surrounding noise level 
#             r.adjust_for_ambient_noise(source2, duration=0.2)
            
#             #listens for the user's input 
#             audio2 = r.listen(source2)
            
#             # Using google to recognize audio
#             MyText = r.recognize_google(audio2)
#             MyText = MyText.lower()

#             print("Did you say ", MyText)
#             SpeakText(MyText)
            
#     except sr.RequestError as e:
#         print("Could not request results; {0}".format(e))
        
#     except sr.UnknownValueError:
#         print("unknown error occurred")


<h4>2. That was surprisingly quick. Now lets make an agent. </h4>
<p>The main thing I want to do right now is build a really simple agent, which only has the "Start", "Agent" and "Output" state. The Agent will take in the user's input as speech, and give its own response in an audio output

In [ ]:
from typing import TypedDict, List
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage
from langgraph.graph import START, END, StateGraph

class AgentState(TypedDict):
    messages: List[HumanMessage]

llm = ChatOllama(model="qwen2.5:0.5b")

def process_node(state: AgentState) -> AgentState:
    """This node simply sends the user input as a human message to chat ollama llm, and prints its response without recording it in the state"""
    response = llm.invoke(state["messages"])
    print("\nAI: " + response.content)
    return state

graph = StateGraph(AgentState)
graph.add_node("Agent", process_node)
graph.add_edge(START, "Agent")
graph.add_edge("Agent", END)

app = graph.compile()
from IPython.display import Image, display
display(Image(app.get_graph().draw_mermaid_png()))

In [ ]:
conversation_history = []

def get_voice_input():
    try:
        
        # use the microphone as source for input.
        with sr.Microphone() as source2:
            
            # wait for a second to let the recognizer adjust the energy threshold based on the surrounding noise level 
            r.adjust_for_ambient_noise(source2, duration=0.2)
            
            #listens for the user's input 
            audio2 = r.listen(source2)
            
            # Using google to recognize audio
            user_input = r.recognize_google(audio2)
            user_input = user_input.lower()
            
    except sr.RequestError as e:
        print("Could not request results; {0}".format())
        
    except sr.UnknownValueError:
        print("unknown error occurred")

    return user_input

command =  get_voice_input()

while command != 'exit game':
     
        print("Human: ", command)
        conversation_history.append(HumanMessage(content=command))
        AiResponse = app.invoke({"messages": conversation_history})
        conversation_history = AiResponse["messages"]
        print(f"{AiResponse["messages"]}")
        #SpeakText(AiResponse)
        command = get_voice_input()

Now lets start to work on making our graph a little more complicated. I am thinking of making a room builder node (setup) and a tool node which turns lights on and off

In [ ]:
# from langchain_core.tools import tool

# @tool
# def house_builder(house : map[map]) -> map[map]:
#     """
#     This function updates the house based on the current input text
#     """
#     house.
# def setup_node(state : AgentState)-> AgentState:
#     """This function is used to ask user for their input on how life works"""

#     return state

<h2>#PART 2 : Full Smart home

In [ ]:
from langgraph.graph import StateGraph, END, START
from langchain_ollama.chat_models import ChatOllama
from langchain_core.messages import HumanMessage, AIMessage
from typing import TypedDict, Dict, List, Union
from langchain_core.tools import tool
import re

# -------------------------------
# STATE
# -------------------------------

class HomeState(TypedDict):
    messages: List[Union[HumanMessage, AIMessage]]
#    home_config: Dict[str, Dict[str, str]]  # room -> {device -> state}


# -------------------------------
# LLM Node: ChatOllama
# -------------------------------

llm = ChatOllama(model="qwen3")

home_config = {}

def agent_node(state: HomeState) -> HomeState:
    """Invoke the local LLM to generate a response based on current message history."""
    response = llm.invoke(state["messages"])
    return {"messages": state["messages"] + [response], "home_config": state["home_config"]}

# -------------------------------
# Setup Node: Build home configuration
# -------------------------------
@tool
def setup_node(state: HomeState) -> HomeState:
    """
    Parse the user's input to initialize room-device configuration.

    Extracts device types and quantities (e.g., '3 lights in the living room') 
    and stores them in a nested map with each device initialized to OFF.
    """
    last_msg = state["messages"][-1]
    if not isinstance(last_msg, HumanMessage):
        return state

    text = last_msg.content.lower()
    room_config = {}
    if any(x in text for x in {"light", "lights", "AC", "ACs", "fan", "fans","TV", "TVs", "speaker", "speakers"}):
        #if "living_room" not in state["home_config"]:
        if "living_room" not in home_config:    
            #state["home_config"]["living_room"] = {}
            home_config["living_room"] = {}

        for i in range(1, 10):
            device_name = f"{x}{i}" 
            #if device_name not in state["home_config"]["living_room"]:
            #    state["home_config"]["living_room"][device_name] = "OFF"
            if device_name not in home_config["living_room"]:
                home_config["living_room"][device_name] = "OFF"
            else:
                continue
    # match = re.findall(r'(\d+)\s+(lights?|fans?|acs?)\s+in\s+the\s+([\w\s]+)', text)

    # for count_str, device_type, room in match:
    #     count = int(count_str)
    #     room = room.strip()
    #     room_key = room.lower()
    #     if room_key not in state["home_config"]:
    #         state["home_config"][room_key] = {}

    #     for i in range(1, count + 1):
    #         device_name = f"{device_type.rstrip('s')}{i}"
    #         state["home_config"][room_key][device_name] = "OFF"

    return state

# -------------------------------
# Operation Node: Turn devices ON/OFF
# -------------------------------

@tool
def operation_node(state: HomeState) -> HomeState:
    """
    Modify the ON/OFF status of devices in specific rooms based on user input.

    Identifies the room and device types mentioned in the message and updates 
    their power state accordingly within the stored home configuration.
    """
    last_msg = state["messages"][-1]
    if not isinstance(last_msg, HumanMessage):
        return state

    text = last_msg.content.lower()
    #room_keys = list(state["home_config"].keys())
    room_keys = home_config.keys()
    for room in room_keys:
        if room in text:
            for device in list(state["home_config"][room].keys()):
                if device.startswith("light") and "light" in text:
                    if "on" in text:
                        home_config[room][device] = "ON"
                    elif "off" in text:
                        home_config[room][device] = "OFF"
                elif device.startswith("fan") and "fan" in text:
                    if "on" in text:
                        home_config[room][device] = "ON"
                    elif "off" in text:
                        home_config[room][device] = "OFF"
                elif device.startswith("ac") and "ac" in text:
                    if "on" in text:
                        home_config[room][device] = "ON"
                    elif "off" in text:
                        home_config[room][device] = "OFF"
    return state

# -------------------------------
# Router Node: Decide next step
# -------------------------------

def router_node(state: HomeState) -> str:
    last_msg = state["messages"][-1]
    if not isinstance(last_msg, HumanMessage):
        return "END"

    text = last_msg.content.lower()
    if "i have" in text or "there are" in text:
        return "SETUP"
    elif "turn on" in text or "turn off" in text:
        return "OPERATE"
    else:
        return "END"

# -------------------------------
# Graph Definition
# -------------------------------

graph = StateGraph(HomeState)

graph.add_node("AGENT", agent_node)
graph.add_node("SETUP", setup_node)
graph.add_node("OPERATE", operation_node)


# Transitions
graph.add_edge(START, "AGENT")
graph.add_conditional_edges("AGENT", router_node, {
    "SETUP": "SETUP",
    "OPERATE": "OPERATE",
    "END": END
})
graph.add_edge("SETUP", "AGENT")
graph.add_edge("OPERATE", "AGENT")


app = graph.compile()


from IPython.display import Image, display
display(Image(app.get_graph().draw_mermaid_png()))


In [ ]:
# -------------------------------
# Run Loop
# -------------------------------

if __name__ == "__main__":
    state: HomeState = {
        "messages": []
    }

home_config = {}

    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit"]:
            break

        state["messages"].append(HumanMessage(content=user_input))
        state = app.invoke(state)
        print(state)
        print("\nUpdated Home Configuration:")
        for room, devices in state["home_config"].items():
            print(f"{room.title()}:")
            for device, status in devices.items():
                print(f"  {device}: {status}")
        print()

Doesn't seem to be working well, hmm, I know, lets get an Open AI model!

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import Runnable, RunnableConfig
from langchain_openai import ChatOpenAI
from datetime import datetime

primary_assistant_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful smart home assistant."
            "You help the user control the electronics in each room of their house"
            " Use the provided tools to create an internal configuration of what the user's house looks like, and also keep a track of which devices are on and off. "
            " When searching, be persistent. Expand your query bounds if the first search returns no results. "
            " If a search comes up empty, expand your search before giving up."
            "\n\nCurrent house structure:\n<User>\n{home_config}\n</User>"
            "\nCurrent time: {time}.",
        ),
        ("placeholder", "{messages}"),
    ]
).partial(time=datetime.now)

tools = [
    setup_node,
    operation_node
]

llm_openai = primary_assistant_prompt | ChatOpenAI(
    llm = "gpt-4o-mini",
    timeout=None,
    temperature=0,
    api_key= API_KEY
).bind_tools(tools)



In [ ]:
from langgraph.checkpoint.memory import MemorySaver

def new_agent_node(state: HomeState) -> HomeState:
    """Invoke the local LLM to generate a response based on current message history."""
    response = llm_openai.invoke(state["messages"])
    return {"messages": state["messages"] + [response]}


builder = StateGraph(HomeState)

builder.add_node("AGENT", new_agent_node)
builder.add_node("SETUP", setup_node)
builder.add_node("OPERATE", operation_node)


# Transitions
builder.add_edge(START, "AGENT")
builder.add_conditional_edges("AGENT", router_node, {
    "SETUP": "SETUP",
    "OPERATE": "OPERATE",
    "END": END
})
builder.add_edge("SETUP", "AGENT")
builder.add_edge("OPERATE", "AGENT")

# memory = MemorySaver()
# test_app = builder.compile(checkpointer=memory)

test_app = builder.compile()

from IPython.display import Image, display
display(Image(test_app.get_graph().draw_mermaid_png()))


In [ ]:
import uuid

test_questions = [
    "Hi there, Can you help me setup my house information?",
    "I have 3 lights in my living room, and 1 AC. I also have 1 light in my bedroom and 1 fan."
    "Can you turn one of the lights in the living room on?",
]

thread_id = str(uuid.uuid4())

# config = {
#     "configurable": {
#         "thread_id": thread_id,
#     }
# }

for question in test_questions:
    response = test_app.invoke(
        {"messages" : ("user", question)},
        # config,
        stream_mode="values"
    )
    print(response)
    print("Updated home configuration : ", home_config)

Never mind, let me start from scratch

In [ ]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

In [ ]:
from typing import TypedDict, Annotated, Sequence, Optional, Dict
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import START, END, StateGraph
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage, SystemMessage, ToolMessage
from langchain_core.tools import tool
from langchain_core.runnables import Runnable,RunnableConfig
from langgraph.graph.message import add_messages # a reducer function which allows merging all data into the state efficiently. If we dont use this reducer function, the updates may replace the existing value entirely!
from langgraph.prebuilt import ToolNode
from langchain_openai import ChatOpenAI


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    config_map : Dict[str, Dict[str, bool]]

@tool
def home_setup(room_key: Optional[str], device_key: Optional[str], config_map: RunnableConfig):
    """This tool takes the current home configuration and the room key and device key as input, and adds the room key to the home configuration"""
    print(config_map)
    if room_key:
        if room_key not in config_map:
            config_map[room_key] = {}

        if device_key:
            if device_key not in config_map[room_key]:
                config_map[room_key][device_key] = False
                return f"The device with ID {device_key} has been succesfully added to the room {room_key} in our home!\n"
        else:
            return f"The room with ID {room_key} has been succesfully addded to our home!\n"
    return "Invalid input. Try again.\n"


@tool
def home_operation(room_key: Optional[str], device_key: Optional[str], config_map: RunnableConfig):
    """This tool takes the already specified room key and device key and the home configuration map as input, and switches the specified device key """
    if not room_key or not device_key:
        return "Invalid input. Keys do not exist."
    print(config_map)
    if room_key not in config_map:
        return "Invalid input. Room does not exist"
    
    if device_key not in config_map[room_key]:
        return "Invalid input. This device has not been set up in this room"
    
    last_operation = "OFF"
    if config_map[room_key][device_key] == False:
        config_map[room_key][device_key] = True
        last_operation = "ON"
    else:
        config_map[room_key][device_key] = False

    return f"The device \"{device_key}\" in room \"{room_key}\" has been turned {last_operation}.\n" 

good_boy_tools = [
    home_setup,
    home_operation
]

good_boy_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful smart home assistant."
            "You help the user control the electronics in each room of their house"
            " Use the provided tools to create an internal configuration of what the user's house looks like, and also keep a track of which devices are on and off. "
            " When searching, be persistent. Expand your query bounds if the first search returns no results. "
            " If a search comes up empty, expand your search before giving up."
            "\n\nCurrent house structure:\n<User>\n{home_config}\n</User>"
        ),
    ]
)

In [ ]:


# model = good_boy_prompt | ChatOllama(model="qwen3", temperature=0).bind_tools(good_boy_tools)
# model = good_boy_prompt | ChatOllama(model="qwen2.5:0.5b", temperature=0).bind_tools(good_boy_tools)
model = good_boy_prompt | ChatOpenAI(model = "gpt-4o-mini", temperature=0).bind_tools(good_boy_tools)
# NODES -

good_boy_home_config = {}

def call_llm(state: State) -> State:
    """This function invokes the llm and gives it basic context"""
    response = model.invoke(state["messages"])
    return {"messages": [response]}

def should_continue(state: State):
    """This function decides if the agent should call a tool or if it should terminate the program"""
    messages = state["messages"]
    lastMessage = messages[-1]
    if not lastMessage.tool_calls:
        return "exit"
    else:
        return "continue"


In [ ]:
    
from langgraph.prebuilt import ToolNode
from langgraph.errors import GraphRecursionError

#GRAPH SETUP
good_boy_graph = StateGraph(State)

good_boy_graph.add_node("Thinker", call_llm)
good_boy_graph.add_node("Doer", ToolNode(tools=good_boy_tools))

good_boy_graph.add_edge(START, "Thinker")
good_boy_graph.add_conditional_edges(
    "Thinker",
    should_continue,
    {
        "exit": END,
        "continue": "Doer"
    }
)

good_boy_graph.add_edge("Doer", "Thinker")

good_boy_app = good_boy_graph.compile()


from IPython.display import Image, display
display(Image(good_boy_app.get_graph().draw_mermaid_png()))

In [ ]:
def print_messages(messages):
    """Function made to print the messages in a more readable format"""
    if not messages:
        return
    
    for message in messages[-3:]:
        if isinstance(message, ToolMessage):
            print(f"\nTOOL RESULT: {message.content}")

def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

inputs = {"messages": [("user", "Hi, I have a living room with 1 AC and 2 lights. Can you turn the AC on for me?")], "config_map": {}}
print_stream(good_boy_app.stream(inputs, stream_mode="values"))

In [ ]:
inputs = {"messages": [("user", "Hi, my room key is : living room and it contains 2 devices with keys light1 and AC1. Can you turn the AC on for me?")], "config_map": {}}
# try:
#     response = good_boy_app.invoke(inputs, {"recursion_limit": 5}, stream_mode="values")
# except GraphRecursionError:
#     print("Recursion Error")
# print(response)

response = good_boy_app.invoke(inputs, stream_mode="values")

In [ ]:
print(response)